<font size=10>**Pràctica 2**</font>




Lluís Picornell Company<br/>
43477189J<br/>
Xavier Vives Marcus<br/>
43462573W<br/>

# Totes les llibreries que emprarem:

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
import numpy as np

# Neteja de dades

In [ ]:
# Llegim dades
data = pd.read_csv('dades.csv')
print(data.columns.values)

Llevam columnes 'PassengerId' , 'Name' i 'Ticket' Ja que no aporten cap informació

In [ ]:
data.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
print(data.columns.values)
data

Podem veure que la columna 'Sex' té valors que no seran compatibles amb el que volem fer. Per tant els substiuim per valors 0 o 1

In [ ]:
data['Sex'].replace('female', 0,inplace=True)
data['Sex'].replace('male', 1,inplace=True)

Comprovem quants de NaNs hi ha al nostre df

In [ ]:
print(data.info())

Podem veure que la columna 'Cabin' només hi ha 204 registres (d'un total de 891) que no son NaNs per tant eliminam la columna 'Cabin'

In [ ]:
data.drop(['Cabin'],axis=1,inplace=True)

Substituim els valors NaN de la columna 'Age' per la mitjana de Age perquè tampoc n'hi ha tants que siguin NaN

In [ ]:
data['Age'].fillna(data['Age'].mean(), inplace = True)

Canviam la columna Embarked per Embarked_C , Embarked_S i Embarked_Q i així poder treballar amb aquesta columna

In [ ]:
clb = data.pop("Embarked")
ohe_clb = pd.get_dummies(clb, prefix='Embarked')
data = pd.concat([data.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)

Podem veure que ja no tenim valors no numèrics

In [ ]:
print(data.info())

Anem a veure com de relacionades estan les dades

In [ ]:
sns.heatmap(data.corr(),annot=True,cmap='crest',linewidths=0.2)
fig=plt.gcf()
fig.set_size_inches(20,12)
plt.show()

Podem veure que 'SibSp' , 'Parch' i 'Age' no estan gens relacionades amb 'Survive' (que és la que ens interessa).
per tant, les llevam.

In [ ]:
data.drop(['SibSp','Parch','Age'],axis=1,inplace=True)
data

Dividim entre dades per entrenar el model i dades per comprovar si funciona

In [ ]:
data_copia=data
all_features = data.drop("Survived",axis=1)
Targeted_feature = data["Survived"]
x_train,x_test,y_train,y_test = train_test_split(all_features,Targeted_feature,test_size=0.33,random_state=69420)

# Regressió Logística

Cream el model i l'entrenam

In [ ]:
modellr = LogisticRegression()
modellr.fit(x_train,y_train)

El testejam i obtenim els resultats

In [ ]:
prediction_lr = modellr.predict(x_test)
print(classification_report(y_test, prediction_lr))

## Feature Importance

Evaluem la importància que li dona a cada columna el model de regressió logística.

In [ ]:
importances = modellr.coef_

importances=str(importances.tolist())
dissallowed_chars="[] "
for c in dissallowed_chars:
    importances=importances.replace(c,"")

importances=importances.split(",")

features=x_train.columns
for i in range(len(features)):
    print("Característica: ",x_train.columns[i],  "\t\tImportància: ",importances[i])

Podem veure que li dona moltíssima importància al sexe i té sentit perquè es deia d'evacuar dones i infants. Per tant, recuperarem la columna 'Age' i mirarem si millora el resultat.

In [ ]:
# Recuperam columna 'Age'
filr=pd.read_csv('dades.csv')
filr.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
filr['Sex'].replace('female', 0,inplace=True)
filr['Sex'].replace('male', 1,inplace=True)
filr.drop(['Cabin'],axis=1,inplace=True)
filr['Age'].fillna(filr['Age'].mean(), inplace = True)
clb = filr.pop("Embarked")
ohe_clb = pd.get_dummies(clb, prefix='Embarked')
filr = pd.concat([filr.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)
filr.drop(['SibSp','Parch'],axis=1,inplace=True)

Tornam a crear el model però ara amb la columna 'Age' i l'entrenam

In [ ]:
all_feat_filr = filr.drop("Survived",axis=1)
Tg_feature_filr = filr["Survived"]
x_tr_filr,x_te_filr,y_tr_filr,y_te_filr = train_test_split(all_feat_filr,Tg_feature_filr,test_size=0.33,random_state=69420)
modelfilr = LogisticRegression(max_iter=200)
modelfilr.fit(x_tr_filr,y_tr_filr)

Obtenim els resultats amb el nou model

In [ ]:
prediction_filr = modelfilr.predict(x_te_filr)
print(classification_report(y_te_filr, prediction_filr))

Veim que tenim els mateixos resultats que sense emprar la columna 'Age'. I això té sentit perquè com hem vist abans, 'Age' no tenia gaire correlació amb 'Survived' per tant, dividirem 'Age' en diferents rangs d'edat a veure si així obtenim un millor resultat.

In [ ]:
filr['Age_bin'] = pd.cut(filr['Age'], bins=[0,7,12,20,40,999], labels=['Infant','Children','Teenage','Adult','Elder'])
filr.drop(['Age'],axis=1,inplace=True)

clb = filr.pop("Age_bin")
ohe_clb = pd.get_dummies(clb, prefix='Age_bin')
filr = pd.concat([filr.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)

all_feat_filr = filr.drop("Survived",axis=1)
Tg_feature_filr = filr["Survived"]
x_tr_filr,x_te_filr,y_tr_filr,y_te_filr = train_test_split(all_feat_filr,Tg_feature_filr,test_size=0.33,random_state=69420)
modelfilr = LogisticRegression(max_iter=2000)
modelfilr.fit(x_tr_filr,y_tr_filr)

Obtenim els resultats amb la nova columna 'Age' diferenciada per rangs d'edat

In [ ]:
prediction_filr = modelfilr.predict(x_te_filr)
print(classification_report(y_te_filr, prediction_filr))

Podem veure que milloram en un 2% respecte a quan no teniem la columna 'Age' 

# Perceptró

In [ ]:
modelp = Perceptron()
modelp.fit(x_train,y_train)
prediction_p = modelp.predict(x_test)
print(classification_report(y_test, prediction_p,zero_division=1))

## Feature Importance

Evaluem la importància que li dona a cada columna el model del perceptró.

In [ ]:
importances = modelp.coef_

importances=str(importances.tolist())
dissallowed_chars="[] "
for c in dissallowed_chars:
    importances=importances.replace(c,"")

importances=importances.split(",")

features=x_train.columns
for i in range(len(features)):
    print("Característica: ",x_train.columns[i],  "\t\tImportància: ",importances[i])

Com podem veure li dona molta més importància al sexe que a la resta, igual que la regreció lineal. Igual que la regració lineal recuperarem la columne "age" per la mateixa raó de dones i infants primer.

In [ ]:
# Recuperam columna 'Age'
fip=pd.read_csv('dades.csv')
fip.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
fip['Sex'].replace('female', 0,inplace=True)
fip['Sex'].replace('male', 1,inplace=True)
fip.drop(['Cabin'],axis=1,inplace=True)
fip['Age'].fillna(fip['Age'].mean(), inplace = True)
clb = fip.pop("Embarked")
ohe_clb = pd.get_dummies(clb, prefix='Embarked')
fip = pd.concat([fip.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)
fip.drop(['SibSp','Parch'],axis=1,inplace=True)

Tornam a crear el model però ara amb la columna 'Age' i l'entrenam

In [ ]:
all_feat_fip = fip.drop("Survived",axis=1)
Tg_feature_fip = fip["Survived"]
x_tr_fip,x_te_fip,y_tr_fip,y_te_fip  = train_test_split(all_feat_fip,Tg_feature_fip,test_size=0.33,random_state=69420)
modelfip = Perceptron(max_iter=200)
modelfip.fit(x_tr_fip,y_tr_fip)

Obtenim els resultats amb el nou model

In [ ]:
prediction_fip = modelfip.predict(x_te_fip)
print(classification_report(y_te_fip, prediction_fip,zero_division = 1))

Veim que tenim els mateixos resultats que sense emprar la columna 'Age'. I això té sentit perquè com hem vist abans, 'Age' no tenia gaire correlació amb 'Survived' per tant, dividirem 'Age' en diferents rangs d'edat a veure si així obtenim un millor resultat.

In [ ]:
fip['Age_bin'] = pd.cut(fip['Age'], bins=[0,7,12,20,40,999], labels=['Infant','Children','Teenage','Adult','Elder'])
fip.drop(['Age'],axis=1,inplace=True)

clb = fip.pop("Age_bin")
ohe_clb = pd.get_dummies(clb, prefix='Age_bin')
fip = pd.concat([fip.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)

all_feat_fip = fip.drop("Survived",axis=1)
Tg_feature_fip = fip["Survived"]
x_tr_fip,x_te_fip,y_tr_fip,y_te_fip = train_test_split(all_feat_fip,Tg_feature_fip,test_size=0.33,random_state=69420)
modelfip = Perceptron(max_iter=2000)
modelfip.fit(x_tr_fip,y_tr_fip)

Obtenim els resultats amb la nova columna 'Age' diferenciada per rangs d'edat

In [ ]:
prediction_fip = modelfip.predict(x_te_fip)
print(classification_report(y_te_fip, prediction_fip,zero_division=1))

Veim que tenir la columna "Age", no tenir-la o dividir-la és totalment indiferent, ja que en els tres casos tenim la mateixa accuracy. 
Per lo que concluim que les millores que hem pensat no afectan al perceptró.

# Arbres de decisió

In [ ]:
modelad = RandomForestClassifier(random_state=69420)
modelad.fit(x_train,y_train)
prediction_ad = modelad.predict(x_test)
print(classification_report(y_test, prediction_ad))

## Feature Importance

Evaluem la importància que li dona a cada columna el model d'Arbres de decisió

In [ ]:
importances =modelad.feature_importances_

importances=str(importances.tolist())
dissallowed_chars="[] "
for c in dissallowed_chars:
    importances=importances.replace(c,"")

importances=importances.split(",")

features=x_train.columns
for i in range(len(features)):
    print("Característica: ",x_train.columns[i],  "\t\tImportància: ",importances[i])

Com podem veure a n'aquest model el que és més important és el "Fare", per lo que farem el mateix que amb l'edat i el separarem per car, mitjà i barato.

In [ ]:
data['Fare_bin'] = pd.cut(data['Fare'], bins=[0,10,31,120], labels=['Cheap','avarage','spencive'])
data.drop(['Fare'],axis=1,inplace=True)

clb = data.pop("Fare_bin")
ohe_clb = pd.get_dummies(clb, prefix='Fare_bin')
data = pd.concat([filr.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)

all_feat_filr = data.drop("Survived",axis=1)
Tg_feature_filr = data["Survived"]
x_tr_firc,x_te_firc,y_tr_firc,y_te_firc = train_test_split(all_feat_filr,Tg_feature_filr,test_size=0.33,random_state=69420)
modelfirc = RandomForestClassifier(random_state=69420)
modelfirc.fit(x_tr_firc,y_tr_firc)

Obtenim els resultats amb el nou model

In [ ]:
prediction_firc = modelfirc.predict(x_te_firc)
print(classification_report(y_te_firc, prediction_firc,zero_division=1))

Com podem observar l'accuracy ha millorat en un 3% d'un 0.79 a un 0.82.

# Conclusions